In [ ]:
#@title Load Project

In [ ]:
#@title Load data

import numpy as np
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
import re
from sklearn.preprocessing import MultiLabelBinarizer 
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from tqdm import tqdm
import pickle

import csv
path = '/content/movies_metadata.csv'
dataframe = pd.read_csv(path,delimiter = ',')

In [ ]:
#@title Sample data

dataframe.head()

,genres,original_title,overview
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",Toy Story,"Led by Woody, Andy's toys live happily in his ..."
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",Jumanji,When siblings Judy and Peter discover an encha...
2,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",Grumpier Old Men,A family wedding reignites the ancient feud be...
3,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom..."
4,"[{'id': 35, 'name': 'Comedy'}]",Father of the Bride Part II,Just when George Banks has recovered from his ...


In [ ]:
#@title Text preprocessing

dataframe['genres']=dataframe['genres'].apply(str)

for i in tqdm(range(len(dataframe))):
  dataframe['genres'][i] = re.sub("[^a-zA-Z]"," ",dataframe['genres'][i])

for i in tqdm(range(len(dataframe))):
  dataframe['genres'][i] = dataframe['genres'][i].replace("id","")
  dataframe['genres'][i] = dataframe['genres'][i].replace("name","")


genre =  [a.split() for a in dataframe['genres']]

data = dataframe[['overview']]

data['genre'] = genre

100%|██████████| 45466/45466 [00:29<00:00, 1545.81it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
dataframe['genres']=dataframe['genres'].apply(str)

In [ ]:
for i in tqdm(range(len(dataframe))):
  dataframe['genres'][i] = re.sub("[^a-zA-Z]"," ",dataframe['genres'][i])

100%|██████████| 45466/45466 [00:15<00:00, 2996.55it/s]


In [ ]:
for i in tqdm(range(len(dataframe))):
  dataframe['genres'][i] = dataframe['genres'][i].replace("id","")
  dataframe['genres'][i] = dataframe['genres'][i].replace("name","")
  
  

100%|██████████| 45466/45466 [00:30<00:00, 1509.22it/s]


In [ ]:
dataframe['genres']

0                       Animation                  Come...
1                       Adventure                  Fant...
2                          Romance                  Com...
3                       Comedy                  Drama  ...
4                                                Comedy   
                               ...                        
45461                   Drama                     Famil...
45462                                             Drama   
45463                   Action                  Drama  ...
45464                                                     
45465                                                     
Name: genres, Length: 45466, dtype: object

In [ ]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   genres          45466 non-null  object
 1   original_title  45466 non-null  object
 2   overview        44512 non-null  object
dtypes: object(3)
memory usage: 1.0+ MB


In [ ]:
genre =  [a.split() for a in dataframe['genres']]

In [ ]:
data = dataframe[['overview']]

In [ ]:
data['genre'] = genre

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
data.dropna(inplace=True)
data.head()

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,overview,genre
0,"Led by Woody, Andy's toys live happily in his ...","[Animation, Comedy, Family]"
1,When siblings Judy and Peter discover an encha...,"[Adventure, Fantasy, Family]"
2,A family wedding reignites the ancient feud be...,"[Romance, Comedy]"
3,"Cheated on, mistreated and stepped on, the wom...","[Comedy, Drama, Romance]"
4,Just when George Banks has recovered from his ...,[Comedy]


In [ ]:
multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(data['genre'])
y = multilabel_binarizer.transform(data['genre'])

In [ ]:
xtrain, xval, ytrain, yval = train_test_split(data['overview'], y, test_size=0.2, random_state=9)

In [ ]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=10000)
xtrain_tfidf = tfidf_vectorizer.fit_transform(xtrain)
xval_tfidf = tfidf_vectorizer.transform(xval)

In [ ]:

clf = OneVsRestClassifier(BernoulliNB())
clf.fit(xtrain_tfidf.toarray(), ytrain)
y_pred = clf.predict(xval_tfidf.toarray())

/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:80: UserWarning: Label not 5 is present in all training examples.
  "Label %s is present in all training examples." % str(classes[c])
/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:80: UserWarning: Label not 6 is present in all training examples.
  "Label %s is present in all training examples." % str(classes[c])
/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:80: UserWarning: Label not 12 is present in all training examples.
  "Label %s is present in all training examples." % str(classes[c])
/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:80: UserWarning: Label not 16 is present in all training examples.
  "Label %s is present in all training examples." % str(classes[c])
/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:80: UserWarning: Label not 20 is present in all training examples.
  "Label %s is present in all training examples." % str(classes[c])
/usr/local/lib/python3

In [ ]:
len(multilabel_binarizer.inverse_transform(y_pred))

8903

In [ ]:
string = ["It has been eight years since Batman (Christian Bale), in collusion with Commissioner Gordon (Gary Oldman), vanished into the night. Assuming responsibility for the death of Harvey Dent, Batman sacrificed everything for what he and Gordon hoped would be the greater good. However, the arrival of a cunning cat burglar (Anne Hathaway) and a merciless terrorist named Bane (Tom Hardy) force Batman out of exile and into a battle he may not be able to win."]
string

['It has been eight years since Batman (Christian Bale), in collusion with Commissioner Gordon (Gary Oldman), vanished into the night. Assuming responsibility for the death of Harvey Dent, Batman sacrificed everything for what he and Gordon hoped would be the greater good. However, the arrival of a cunning cat burglar (Anne Hathaway) and a merciless terrorist named Bane (Tom Hardy) force Batman out of exile and into a battle he may not be able to win.']

In [ ]:
tfidf_vectorizer.transform(string).toarray()

array([[0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
multilabel_binarizer.inverse_transform(clf.predict(tfidf_vectorizer.transform(string).toarray()))

[('Action', 'Animation', 'Crime', 'Thriller')]

In [ ]:
pickle.dump(clf,open('movie_model.pkl','wb'))

In [ ]:
pickle.dump(multilabel_binarizer,open('target_label.pkl','wb'))
pickle.dump(tfidf_vectorizer,open('vectorizer.pkl','wb'))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
